In [1]:
import pandas as pd
import psycopg2
# Use psycopg2's execute_values for efficient bulk insert
from psycopg2.extras import execute_values
from psycopg2 import sql

In [2]:
# Database connection parameters (replace with actual credentials)
db_params = {
    'dbname': 'smog_db',
    'user': 'smog_dev',
    'password': 'kcSzXTAg1RSXktipRM9gN9cD32pGFN2k',
    'host': 'dpg-csnu6npu0jms73913fh0-a.oregon-postgres.render.com',
    'port': '5432'
}

In [3]:
# Establish the connection
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cursor = conn.cursor()

In [4]:
# Function to list all tables
def list_tables():
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

In [18]:
list_tables()

Tables in the database:
combined_hourly_data
octapi_data


In [8]:
data_list = ["June2024.csv", "July2024.csv", "August2024.csv", "September2024.csv", "October2024.csv", "November2024.csv"]
import pandas as pd

# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through each file in the data_list
for file in data_list:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames vertically
combined_data = pd.concat(dataframes, ignore_index=True)


In [9]:
combined_data.shape

(996951, 15)

In [10]:
combined_data.head(10)

,District,date,Vehicle,Industry,Residential,Misc,Construction,Agriculture,Sum_of_Sources,Vehicle%,Industry%,Residential%,Misc%,Construction%,Agriculture%
0,Khushab,6/1/2024 0:00,145.030000,76.430000,31.215,12.910,11.605000,258.910000,536.099999,27.052789,14.256669,5.822608,2.408133,2.164708,48.295094
1,Sialkot,6/1/2024 0:00,235.170000,110.905000,49.135,0.967,8.615000,97.008000,501.800001,46.865285,22.101435,9.791750,0.192706,1.716819,19.332005
2,Bahawalnagar,6/1/2024 0:00,130.365000,119.415000,34.750,62.730,72.460000,430.079999,849.799999,15.340668,14.052130,4.089197,7.381737,8.526712,50.609555
3,Rawalpindi,6/1/2024 0:00,245.150000,114.775000,55.080,4.140,15.900000,98.755000,533.799999,45.925440,21.501499,10.318471,0.775571,2.978644,18.500375
4,Dera Ghazi Khan,6/1/2024 0:00,151.290000,69.090000,38.760,24.830,8.810000,272.220000,565.000000,26.776991,12.228319,6.860177,4.394690,1.559292,48.180531
5,Bahawalpur,6/1/2024 0:00,101.500001,107.870001,36.975,170.640,113.880001,815.535003,1346.400007,7.538622,8.011735,2.746212,12.673797,8.458111,60.571524
6,Faisalabad,6/1/2024 0:00,430.010000,300.434999,12.500,9.102,43.810000,212.843000,1008.699998,42.630118,29.784376,1.239219,0.902350,4.343214,21.100724
7,Chakwal,6/1/2024 0:00,184.830000,58.100000,49.510,50.900,8.300000,146.360000,498.000000,37.114458,11.666667,9.941767,10.220884,1.666667,29.389558
8,Sargodha,6/1/2024 0:00,171.100000,112.680000,34.885,7.920,10.980000,212.935000,550.499999,31.080836,20.468665,6.336966,1.438692,1.994550,38.680291
9,Sargodha,6/1/2024 0:00,135.140000,88.110000,29.180,8.100,10.850000,156.020000,427.399998,31.619092,20.615349,6.827328,1.895180,2.538605,36.504446


In [11]:
combined_data.duplicated().sum()

336921

In [12]:
combined_data.tail(10)

,District,date,Vehicle,Industry,Residential,Misc,Construction,Agriculture,Sum_of_Sources,Vehicle%,Industry%,Residential%,Misc%,Construction%,Agriculture%
996941,Rahim Yar Khan,11/21/2024 23:00,205.885000,100.700000,36.185,2.318,19.070000,286.541999,650.699999,31.640541,15.475642,5.560934,0.356232,2.930690,44.035961
996942,Lodhran,11/21/2024 23:00,446.649999,122.290000,121.240,40.765,38.070000,1643.884997,2412.899996,18.510920,5.068175,5.024659,1.689461,1.577769,68.129015
996943,Sheikhupura,11/21/2024 23:00,1086.284999,518.045000,203.110,9.347,61.500000,323.113000,2201.399998,49.345189,23.532525,9.226401,0.424593,2.793677,14.677614
996944,Gujrat,11/21/2024 23:00,283.810001,242.570002,81.195,13.785,19.480001,232.060001,872.900004,32.513461,27.788979,9.301753,1.579219,2.231642,26.584947
996945,Attock,11/21/2024 23:00,408.505000,125.765000,62.030,19.120,9.760000,608.020000,1233.200000,33.125608,10.198265,5.030003,1.550438,0.791437,49.304249
996946,Muzaffargarh,11/21/2024 23:00,444.685000,318.060000,83.700,24.190,35.885000,682.280000,1588.800001,27.988734,20.018882,5.268127,1.522533,2.258623,42.943102
996947,Khushab,11/21/2024 23:00,291.035000,137.965000,66.015,35.045,33.760000,684.080000,1247.900001,23.321981,11.055774,5.290087,2.808318,2.705345,54.818495
996948,Bahawalpur,11/21/2024 23:00,70.010000,37.660000,19.175,1.085,1.870000,176.100000,305.900001,22.886564,12.311213,6.268388,0.354691,0.611311,57.567833
996949,Sheikhupura,11/21/2024 23:00,1283.239999,585.299997,238.560,9.032,59.339999,375.927999,2551.399994,50.295524,22.940346,9.350161,0.354002,2.325782,14.734185
996950,Muzaffargarh,11/21/2024 23:00,375.510000,260.350000,70.650,15.060,22.090000,548.339999,1291.999998,29.064242,20.150929,5.468266,1.165635,1.709752,42.441176


In [14]:
# Reorder the columns to make 'date' the first column
combined_data = combined_data[['date'] + [col for col in combined_data.columns if col != 'date']]

In [15]:
combined_data.head()

,date,District,Vehicle,Industry,Residential,Misc,Construction,Agriculture,Sum_of_Sources,Vehicle%,Industry%,Residential%,Misc%,Construction%,Agriculture%
0,6/1/2024 0:00,Khushab,145.030,76.430,31.215,12.910,11.605,258.910000,536.099999,27.052789,14.256669,5.822608,2.408133,2.164708,48.295094
1,6/1/2024 0:00,Sialkot,235.170,110.905,49.135,0.967,8.615,97.008000,501.800001,46.865285,22.101435,9.791750,0.192706,1.716819,19.332005
2,6/1/2024 0:00,Bahawalnagar,130.365,119.415,34.750,62.730,72.460,430.079999,849.799999,15.340668,14.052130,4.089197,7.381737,8.526712,50.609555
3,6/1/2024 0:00,Rawalpindi,245.150,114.775,55.080,4.140,15.900,98.755000,533.799999,45.925440,21.501499,10.318471,0.775571,2.978644,18.500375
4,6/1/2024 0:00,Dera Ghazi Khan,151.290,69.090,38.760,24.830,8.810,272.220000,565.000000,26.776991,12.228319,6.860177,4.394690,1.559292,48.180531


In [23]:
combined_data.isna().sum()

date              0
District          0
Vehicle           0
Industry          0
Residential       0
Misc              0
Construction      0
Agriculture       0
Sum_of_Sources    0
Vehicle%          0
Industry%         0
Residential%      0
Misc%             0
Construction%     0
Agriculture%      0
dtype: int64

In [107]:
combined_data.columns

Index(['date', 'District', 'Vehicle', 'Industry', 'Residential', 'Misc',
       'Construction', 'Agriculture', 'Sum_of_Sources', 'Vehicle%',
       'Industry%', 'Residential%', 'Misc%', 'Construction%', 'Agriculture%'],
      dtype='object')

In [16]:
columns = combined_data.columns
columns

Index(['date', 'District', 'Vehicle', 'Industry', 'Residential', 'Misc',
       'Construction', 'Agriculture', 'Sum_of_Sources', 'Vehicle%',
       'Industry%', 'Residential%', 'Misc%', 'Construction%', 'Agriculture%'],
      dtype='object')

In [94]:
columns = ['date', 'District', 'Vehicle', 'Industry', 'Residential', 'Misc',
       'Construction', 'Agriculture', 'Sum_of_Sources', 'Vehicle_percentage',
       'Industry_percentage', 'Residential_percentage', 'Misc_percentage', 'Construction_percentage', 'Agriculture_percentage']


In [92]:
# # SQL to drop the table if it exists
# drop_table_query = "DROP TABLE IF EXISTS octapi_data;"

# # Execute the query
# cursor.execute(drop_table_query)
# conn.commit()

# print("Table 'octapi_data' dropped (if it existed).")


Table 'octapi_data' dropped (if it existed).


In [95]:
create_table_query = sql.SQL(
    "CREATE TABLE IF NOT EXISTS octapi_data ("
    '"date" DATE, '
    '"District" TEXT, '  # District as TEXT for string data
    + ", ".join([f'"{col}" FLOAT' for col in columns[2:]]) + ");"
)

# Execute the query
cursor.execute(create_table_query)
conn.commit()
print("Table created successfully.")


Table created successfully.


In [96]:
# Fetch the structure of the newly created table
cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'octapi_data';")
table_structure = cursor.fetchall()

print("Table Structure:")
for column in table_structure:
    print(column)


Table Structure:
('Agriculture_percentage', 'double precision')
('date', 'date')
('Vehicle', 'double precision')
('Industry', 'double precision')
('Residential', 'double precision')
('Misc', 'double precision')
('Construction', 'double precision')
('Agriculture', 'double precision')
('Sum_of_Sources', 'double precision')
('Vehicle_percentage', 'double precision')
('Industry_percentage', 'double precision')
('Residential_percentage', 'double precision')
('Misc_percentage', 'double precision')
('Construction_percentage', 'double precision')
('District', 'text')


In [97]:
list_tables()

Tables in the database:
combined_hourly_data
octapi_data


In [56]:
combined_data.isna().sum()

date              0
District          0
Vehicle           0
Industry          0
Residential       0
Misc              0
Construction      0
Agriculture       0
Sum_of_Sources    0
Vehicle%          0
Industry%         0
Residential%      0
Misc%             0
Construction%     0
Agriculture%      0
dtype: int64

In [98]:
values = [tuple(row) for row in combined_data.to_numpy()]

In [99]:
values[:2]

[('6/1/2024 0:00',
  'Khushab',
  145.0299999,
  76.4299999,
  31.21499994,
  12.90999982,
  11.60499983,
  258.9099996,
  536.099999,
  27.05278869,
  14.25666854,
  5.822607722,
  2.408132783,
  2.164708049,
  48.29509421),
 ('6/1/2024 0:00',
  'Sialkot',
  235.1700002,
  110.9050002,
  49.13500004,
  0.967000003,
  8.615000057,
  97.00800012,
  501.8000007,
  46.86528496,
  22.10143485,
  9.791749696,
  0.192706258,
  1.716819459,
  19.33200478)]

In [115]:
columns

['date',
 'District',
 'Vehicle',
 'Industry',
 'Residential',
 'Misc',
 'Construction',
 'Agriculture',
 'Sum_of_Sources',
 'Vehicle_percentage',
 'Industry_percentage',
 'Residential_percentage',
 'Misc_percentage',
 'Construction_percentage',
 'Agriculture_percentage']

In [100]:
insert_query = sql.SQL(
    "INSERT INTO octapi_data ({}) VALUES %s"
).format(
    sql.SQL(", ").join([sql.Identifier(col) for col in columns])
)


In [101]:
# Establish the connection
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cursor = conn.cursor()

In [102]:
try:
    execute_values(cursor, insert_query, values)
    conn.commit()
    print("Data inserted successfully.")
except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()


Data inserted successfully.


In [76]:
# # Fetch the first column from the table
# cursor.execute("SELECT {} FROM octapi_data".format(sql.Identifier(columns[0])))
# first_column_data = cursor.fetchall()

# # Print the first column data
# print("First column data:", first_column_data)


UndefinedFunction: function identifier(unknown) does not exist
LINE 1: SELECT Identifier('date') FROM octapi_data
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [ ]:
def save_dataframe_to_db(df, table_name, conn_params):
    """
    Save the values of a DataFrame to a specified database table.

    Parameters:
    df (pd.DataFrame): The DataFrame to save.
    table_name (str): The name of the database table.
    conn_params (dict): Database connection parameters.

    Returns:
    None
    """
    # Establish the connection
    conn = psycopg2.connect(**conn_params)
    conn.autocommit = True
    cursor = conn.cursor()

    # Prepare the insert query
    columns = df.columns.tolist()
    insert_query = sql.SQL(
        "INSERT INTO {} ({}) VALUES %s"
    ).format(
        sql.Identifier(table_name),
        sql.SQL(", ").join(map(sql.Identifier, columns))
    )

    # Prepare the values for insertion
    values = [tuple(x) for x in df.to_numpy()]

    try:
        execute_values(cursor, insert_query, values)
        conn.commit()
        print("Data inserted successfully.")
    except Exception as e:
        print(f"Error occurred: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


In [103]:
# Load the Octapi.csv file
octapi_df = pd.read_csv('Octapi.csv')

# Check for duplicates in the DataFrame
duplicates = octapi_df.duplicated()

# Print the number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

# Optionally, print the duplicate rows themselves
if duplicates.any():
    print("Duplicate rows:")
    print(octapi_df[duplicates])


Number of duplicate rows: 553
Duplicate rows:
        District                 date     Vehicle    Industry  Residential  \
1863  Bahawalpur  2024-11-22 15:00:00   86.220000   45.960000    24.515000   
2184     Bhakkar  2024-11-22 00:00:00  249.050000  133.290000    69.030000   
2185     Bhakkar  2024-11-22 01:00:00  241.190000  129.230000    66.910000   
2186     Bhakkar  2024-11-22 02:00:00  227.899999  122.779999    63.585000   
2187     Bhakkar  2024-11-22 03:00:00  220.510000  119.010000    61.915000   
...          ...                  ...         ...         ...          ...   
7651      Lahore  2024-11-22 19:00:00  713.019999  351.319997   121.165000   
7652      Lahore  2024-11-22 20:00:00  698.714999  342.900000   119.610000   
7653      Lahore  2024-11-22 21:00:00  700.305002  339.590002   120.545000   
7654      Lahore  2024-11-22 22:00:00  715.484996  340.844996   123.249999   
7655      Lahore  2024-11-22 23:00:00  745.150003  347.030004   128.120001   

        Misc  Con

In [120]:
import numpy as np

def load_and_group_by_district(input_date: str, input_district: str):
    """
    Query the database to filter by date and district, calculate max values for each column, 
    and return the result as a dictionary.
    """
    try:
        # Ensure the input date is a datetime object
        input_date = pd.to_datetime(input_date).strftime('%Y-%m-%d')  # Format to YYYY-MM-DD for SQL
        
        # Connect to the database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        
        # Query the database for the specified date and district
        query = """
        SELECT * FROM octapi_data
        WHERE "date" = %s AND "District" = %s;
        """
        cursor.execute(query, (input_date, input_district))
        rows = cursor.fetchall()
        
        # Get column names from the database
        cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'octapi_data';")
        column_names = ['date',
                        'District',
                        'Vehicle',
                        'Industry',
                        'Residential',
                        'Misc',
                        'Construction',
                        'Agriculture',
                        'Sum_of_Sources',
                        'Vehicle_percentage',
                        'Industry_percentage',
                        'Residential_percentage',
                        'Misc_percentage',
                        'Construction_percentage',
                        'Agriculture_percentage']
        
        print(column_names)
        
        # Convert the result to a pandas DataFrame
        filtered_data = pd.DataFrame(rows, columns=column_names)
        
        # Close the connection
        cursor.close()
        conn.close()
        
        # If no data is found, return an empty dictionary
        if filtered_data.empty:
            return {}
        
        # Calculate the maximum value for each column in the filtered data
        max_values = filtered_data.max()
        
        # Drop the 'date' and 'District' columns
        max_values = max_values.drop(['date', 'District'])
        
        # Replace NaN, inf, -inf with None so it's JSON serializable
        max_values.replace([np.inf, -np.inf, np.nan], None, inplace=True)
        
        # Convert the result to a dictionary
        result_dict = max_values.to_dict()
        
        # Replace '_percentage' with '%' in the keys
        result_dict = {key.replace('_percentage', '%'): value for key, value in result_dict.items()}
        
        return result_dict

    except Exception as e:
        # Handle exceptions and return an error message
        return {"error": str(e)}


In [121]:
test = load_and_group_by_district("2024-11-22 00:00:00", "Khushab")

['date', 'District', 'Vehicle', 'Industry', 'Residential', 'Misc', 'Construction', 'Agriculture', 'Sum_of_Sources', 'Vehicle_percentage', 'Industry_percentage', 'Residential_percentage', 'Misc_percentage', 'Construction_percentage', 'Agriculture_percentage']


In [122]:
test

{'Vehicle': 415.64499979019166,
 'Industry': 195.4349997520447,
 'Residential': 92.0649998664856,
 'Misc': 45.46999950408935,
 'Construction': 44.25499954223633,
 'Agriculture': 962.2149986267088,
 'Sum_of_Sources': 1732.299997329712,
 'Vehicle%': 28.250681116119235,
 'Industry%': 15.559945481570772,
 'Residential%': 6.352861020853415,
 'Misc%': 3.8814814441435765,
 'Construction%': 3.7071224670425016,
 'Agriculture%': 56.61087252775994}

In [116]:
columns

['date',
 'District',
 'Vehicle',
 'Industry',
 'Residential',
 'Misc',
 'Construction',
 'Agriculture',
 'Sum_of_Sources',
 'Vehicle_percentage',
 'Industry_percentage',
 'Residential_percentage',
 'Misc_percentage',
 'Construction_percentage',
 'Agriculture_percentage']